### $$\textbf{Housing ads analysis}$$ 
House owners or rental agencies, hereafter dealers, usually publish their ads in more than one portal for better visibility. The purpose of this script is to analyse the attached dataset to determine ads published in different portal but referring to the same property.

<b>1. Importing relevant libraries

In [2]:
import pandas as pd

from fuzzywuzzy import fuzz # Python library allowing to compare text/sentences in order to determine the level of similarity

<b>2. Data preparation

In [3]:
# Importing the dataset

data = pd.read_excel("C:/Users/ilung/Desktop/housing_ad/dataset.xlsx")

In [4]:
# Remplacing all NaN (rows without information) in "DEALER_NAME" column par "unknown dealer". 

data[["DEALER_NAME"]] = data[["DEALER_NAME"]].fillna("unknown dealer")

In [5]:
# Keeping the most important columns which are "ID", "URL", "DESCRIPTION" et "DEALER_NAME"

data_cleaned = data[["ID","URL","DESCRIPTION", "DEALER_NAME"]]

<b> 3. Creating a dataframe containing dealers with only one ads

In [6]:
# Deleting dealers that are repeated more than once and only keeping those with a single occurrence

df_single = data_cleaned.drop_duplicates(subset = ["DEALER_NAME"], keep= False) 

df_single.to_excel("C:/Users/ilung/Desktop/housing_ad/single_ad.xlsx", index = False)

<b> 4. Creating a dataframe containing dealers with multiple ads

In [7]:
# To produce this dataframe, we will remove the ads identified in step 3 from the cleaned_dataframe. We will use their index to achieve this

df_single_index_list = df_single.index.values.tolist()

df_multi = data_cleaned.drop(index = df_single_index_list)

df_multi.to_excel("C:/Users/ilung/Desktop/housing_ad/multi_ad.xlsx", index = False)


From now on, we will be working with the dataframe containing dealers with multiple ads i.e "df_multi"

<b> 5. Generating a list of dealers

In [8]:
list_dealer = list(set(list(df_multi["DEALER_NAME"]))) 

<b> 6. Determining ads that refer to the same property for each dealer

In [9]:
for position in list(range(len(list_dealer))): # This will iterate over the list of dealers

    df = df_multi.loc[df_multi["DEALER_NAME"] == list_dealer[position]]

    # Here starts the comparison of ads for each dealer
    
    fuzz_results = [] # Here will be stored the lists of ID of matching ads. Each list will contain pair of matching ads because fuzzywuzzy can only compared elements two by two

    for i in list(range(len(df))):

        j = i + 1

        while j < len(df):

            str1 = df.iloc[i]['DESCRIPTION']
    
            str2 = df.iloc[j]['DESCRIPTION']
    
            theratio = fuzz.token_set_ratio(str1, str2) # fuzzywuzzy can only compared items two by two

            if theratio > 90:

                thepair = list((df.iloc[i]['ID'], df.iloc[j]['ID']))

                fuzz_results.append(thepair)
        
            j = j + 1

    # Using the fuzz_results, we will group ads referring to the same property using the transitive property of equality
            
    ads_grouping = [] # This list will contain dictionnaries. Each dictionnary will contain the ID of ads referring to the same property

    while len(fuzz_results) > 0 :

        first, *rest = fuzz_results
        first = set(first)

        lf = -1
        while len(first)>lf:
            
            lf = len(first)

            rest2 = []
            for r in rest:
                if len(first.intersection(set(r)))>0:
                    first |= set(r)
                else:
                    rest2.append(r)     
            rest = rest2

        ads_grouping.append(first)

        fuzz_results = rest
    
    print ("RESULTS FOR ", list_dealer[position]) # This is the name of dealers

    print(ads_grouping)

# The printed results will display the ID of properties ads
# Empty list means that the dealer ads refer to different properties

RESULTS FOR  Consultants Immobilier Wagram
[{'18224720-a3f7-11e9-aa5e-8b8909b4f047', 'acb994d0-a32d-11e9-96d9-fb3253f8fc6c'}]
RESULTS FOR  Bnppre Bureaux Paris Location < 5000 M²
[]
RESULTS FOR  Valopark
[{'b70641a0-9bd5-11e8-9715-3760d659752c', '46d07c70-9b44-11e8-9715-3760d659752c', 'b0daaaa0-9bd5-11e8-9715-3760d659752c'}]
RESULTS FOR  Terrasses Et Jardins
[{'90adb0e0-75c5-11e9-8084-55ce2049f05c', '1e263050-77a1-11e9-8084-55ce2049f05c', 'a7310700-75fb-11e9-8084-55ce2049f05c', '5f0c9450-75ae-11e9-adc1-59e67c527828'}]
RESULTS FOR  BUILDING PARTNERS LEVALLOIS PERRET
[{'e2040f90-3491-11e9-8213-25cc7d9bf5fc', 'd8368ef0-67b0-11e9-adc1-59e67c527828', '8c0f34d0-1465-11e9-8321-1350c9447565', 'dbe80e20-f294-11e8-9c01-a577da129fbb', 'e66bd590-0575-11e9-8321-1350c9447565', '0d69c050-9e99-11e9-a8e6-0de7b497e456', '0312e330-8971-11e9-8a42-a16638394d93', 'cb916400-78f3-11e9-84a7-e58cae822292', 'd489daa0-67b0-11e9-8084-55ce2049f05c', 'd2b6ff50-f294-11e8-9c01-a577da129fbb', '49759520-7698-11e9-adc1-5